#load taxi journeys
> will take ~15 mins <br>
> requires hadoop-aws e.g. org.apache.hadoop:hadoop-aws:3.0.1

In [0]:
spark.conf.set('fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')

In [0]:
nys_taxi_s3_template = 's3a://nyc-tlc/trip data/{t}_tripdata_{yyyy}-{mm}.csv'
types = ['yellow', 'green', 'fhv']
years = ['2019', '2018']  # 2020 processed differently below
months = [str(i).zfill(2) for i in range(1,13)]

In [0]:
%sql 
DROP TABLE IF EXISTS taxi_journeys


In [0]:
%sql
DROP TABLE IF EXISTS taxi_journeys_sm

In [0]:
type_changes = [
  ('tpep_pickup_datetime', 'timestamp'),
  ('tpep_dropoff_datetime', 'timestamp'),
  ('passenger_count', 'int'),
  ('trip_distance', 'double'),
  ('fare_amount', 'double'),
  ('extra', 'double'),
  ('mta_tax', 'double'),
  ('tip_amount', 'double'),
  ('tolls_amount', 'double'),
  ('improvement_surcharge', 'double'),
  ('total_amount', 'double'),
  ('congestion_surcharge', 'double'),
]

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
window_spec = Window.orderBy('tpep_pickup_datetime')

In [0]:
yellow_2020_csvs = [nys_taxi_s3_template.format(t='yellow', yyyy=2020, mm=mm) for mm in months[:6]]  # 2020 only recorded until june
yellow_2019_2018_csvs = [nys_taxi_s3_template.format(t='yellow', yyyy=yyyy, mm=mm) for yyyy in years for mm in months]
yellow_2020_2018_csvs = yellow_2020_csvs + yellow_2019_2018_csvs
yellow_df = spark.read.csv(yellow_2020_2018_csvs, header=True)

for colname, t in type_changes:
  yellow_df = yellow_df.withColumn(colname, yellow_df[colname].cast(t))
  
yellow_df = yellow_df.withColumn('idx', row_number().over(window_spec))
yellow_df.write.saveAsTable('taxi_journeys')

smaller sampled dataset

In [0]:
sample_df = spark.sql('SELECT * FROM taxi_journeys').sample(fraction=0.1)
sample_df.write.saveAsTable('taxi_journeys_sm')